In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from data_utils.data import CatsAndDogs
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F
from models.classifier import ClassifierWithoutBatchNorm

In [2]:
BATCH_SIZE = 128
ETA = 1e-3
EPOCHS = 30

In [3]:
def compute_accuracy(ps, labels):
    equality = (torch.round(ps) == labels).float()
    acc = equality.mean()
    return acc

In [4]:
def train_step(context, x, y):
    model = context["model"]
    optimizer = context["optimizer"]
    
    optimizer.zero_grad()
    
    ps = model(x)
    ps = ps.squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    loss.backward()
    optimizer.step()
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [5]:
def eval_step(context, x, y):
    model = context["model"]
    
    ps = model(x)
    ps = ps.squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [6]:
def train():
    trainset = CatsAndDogs(mode="train")
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    
    testset = CatsAndDogs(mode="test")
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    model = ClassifierWithoutBatchNorm().cuda()
    optimizer = optim.Adam(model.parameters())
    
    context = {
        "model": model,
        "optimizer": optimizer
    }
    
    for e in range(EPOCHS):
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0
        
        model.train()
        
        for x, y in train_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            loss, acc = train_step(context, x, y)
            train_loss += loss
            train_acc += acc
            
        model.eval()
            
        for x, y in test_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            with torch.no_grad():
                loss, acc = eval_step(context, x, y)
                test_loss += loss
                test_acc += acc
                
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        print(f"Epochs {e+1}/{EPOCHS}")
        print(f"Train loss: {train_loss:.8f}, train acc: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")

In [7]:
train()

Epochs 1/30
Train loss: 0.69472864, train acc: 0.3789
Test loss: 0.69374013, test acc: 0.4355
Epochs 2/30
Train loss: 0.68579343, train acc: 0.5664
Test loss: 0.69469434, test acc: 0.5000
Epochs 3/30
Train loss: 0.66044176, train acc: 0.6510
Test loss: 0.69696128, test acc: 0.5000
Epochs 4/30
Train loss: 0.67356372, train acc: 0.5378
Test loss: 0.70061898, test acc: 0.5000
Epochs 5/30
Train loss: 0.66969237, train acc: 0.5000
Test loss: 0.70363975, test acc: 0.5000
Epochs 6/30
Train loss: 0.67874429, train acc: 0.4622
Test loss: 0.70374274, test acc: 0.5000
Epochs 7/30
Train loss: 0.63129789, train acc: 0.6328
Test loss: 0.70455366, test acc: 0.5161
Epochs 8/30
Train loss: 0.61943698, train acc: 0.6562
Test loss: 0.70851731, test acc: 0.5000
Epochs 9/30
Train loss: 0.62752965, train acc: 0.6758
Test loss: 0.71553910, test acc: 0.4194
Epochs 10/30
Train loss: 0.61310637, train acc: 0.6914
Test loss: 0.72760981, test acc: 0.4194
Epochs 11/30
Train loss: 0.60775223, train acc: 0.7109
Test